### Inference

##### 경로 확인 및 설정

In [1]:
pwd

'c:\\Users\\ADMIN\\Desktop\\dl업로드용\\Kobertsum_0.3v'

In [2]:
cd ./src

c:\Users\ADMIN\Desktop\dl업로드용\Kobertsum_0.3v\src


##### 필요 라이브러리 불러오기

In [3]:
import copy
import torch
import torch.nn as nn
from transformers import BertModel, BertConfig
from torch.nn.init import xavier_uniform_
from transformers import AutoModel, AutoTokenizer

from models.model_builder import *
from models.encoder import *

import argparse
import os
import math
import numpy as np

from transformers import BertTokenizer, BertModel
from prepro.tokenization_kobert import *
from prepro.tokenization_kobert import KoBertTokenizer
from kss import split_sentences

##### 인자 설정

In [4]:
def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

parser = argparse.ArgumentParser()
parser.add_argument("-encoder", default='bert', type=str, choices=['bert', 'baseline'])
parser.add_argument("-bert_data_path", default='../bert_data_new/cnndm')
parser.add_argument("-model_path", default='../models/')
parser.add_argument("-result_path", default='../results/cnndm')
parser.add_argument("-temp_dir", default='../temp')

parser.add_argument("-batch_size", default=140, type=int)
parser.add_argument("-test_batch_size", default=200, type=int)

parser.add_argument("-max_pos", default=512, type=int)
parser.add_argument("-use_interval", type=str2bool, nargs='?',const=True,default=True)
parser.add_argument("-large", type=str2bool, nargs='?',const=True,default=False)
parser.add_argument("-load_from_extractive", default='', type=str)

parser.add_argument("-sep_optim", type=str2bool, nargs='?',const=True,default=False)
parser.add_argument("-lr_bert", default=2e-3, type=float)
parser.add_argument("-lr_dec", default=2e-3, type=float)
parser.add_argument("-use_bert_emb", type=str2bool, nargs='?',const=True,default=False)

parser.add_argument("-share_emb", type=str2bool, nargs='?', const=True, default=False)
parser.add_argument("-finetune_bert", type=str2bool, nargs='?', const=True, default=True)
parser.add_argument("-dec_dropout", default=0.2, type=float)
parser.add_argument("-dec_layers", default=6, type=int)
parser.add_argument("-dec_hidden_size", default=768, type=int)
parser.add_argument("-dec_heads", default=8, type=int)
parser.add_argument("-dec_ff_size", default=2048, type=int)
parser.add_argument("-enc_hidden_size", default=512, type=int)
parser.add_argument("-enc_ff_size", default=512, type=int)
parser.add_argument("-enc_dropout", default=0.2, type=float)
parser.add_argument("-enc_layers", default=6, type=int)

parser.add_argument("-pretrained_model", default='bert', type=str)

parser.add_argument("-mode", default='', type=str)
parser.add_argument("-select_mode", default='greedy', type=str)
parser.add_argument("-map_path", default='../../data/')
parser.add_argument("-raw_path", default='../../line_data')
parser.add_argument("-save_path", default='../../data/')

parser.add_argument("-shard_size", default=2000, type=int)
parser.add_argument('-min_src_nsents', default=1, type=int)    # 3
parser.add_argument('-max_src_nsents', default=120, type=int)    # 100
parser.add_argument('-min_src_ntokens_per_sent', default=1, type=int)    # 5
parser.add_argument('-max_src_ntokens_per_sent', default=300, type=int)    # 200
parser.add_argument('-min_tgt_ntokens', default=1, type=int)    # 5
parser.add_argument('-max_tgt_ntokens', default=500, type=int)    # 500

parser.add_argument("-lower", type=str2bool, nargs='?',const=True,default=True)
parser.add_argument("-use_bert_basic_tokenizer", type=str2bool, nargs='?',const=True,default=False)

parser.add_argument('-log_file', default='../../logs/cnndm.log')

parser.add_argument('-dataset', default='')

parser.add_argument('-n_cpus', default=2, type=int)

# params for EXT
parser.add_argument("-ext_dropout", default=0.2, type=float)
parser.add_argument("-ext_layers", default=2, type=int)
parser.add_argument("-ext_hidden_size", default=768, type=int)
parser.add_argument("-ext_heads", default=8, type=int)
parser.add_argument("-ext_ff_size", default=2048, type=int)

parser.add_argument("-label_smoothing", default=0.1, type=float)
parser.add_argument("-generator_shard_size", default=32, type=int)
parser.add_argument("-alpha",  default=0.6, type=float)
parser.add_argument("-beam_size", default=5, type=int)
parser.add_argument("-min_length", default=15, type=int)
parser.add_argument("-max_length", default=150, type=int)
parser.add_argument("-max_tgt_len", default=140, type=int)

args = parser.parse_args('')

##### BertData 변환

In [5]:
class BertData():
    def __init__(self):
        #self.tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert", do_lower_case=True)
        self.tokenizer = BertTokenizer.from_pretrained("monologg/kobigbird-bert-base")

        self.sep_token = '[SEP]'
        self.cls_token = '[CLS]'
        self.pad_token = '[PAD]'
        # self.sep_vid = self.tokenizer.token2idx[self.sep_token]
        # self.cls_vid = self.tokenizer.token2idx[self.cls_token]
        # self.pad_vid = self.tokenizer.token2idx[self.pad_token]
        self.sep_vid = self.tokenizer.convert_tokens_to_ids(self.sep_token)
        self.cls_vid = self.tokenizer.convert_tokens_to_ids(self.cls_token)
        self.pad_vid = self.tokenizer.convert_tokens_to_ids(self.pad_token)

    def preprocess(self, src):

        if (len(src) == 0):
            return None

        original_src_txt = [' '.join(s) for s in src]
        idxs = [i for i, s in enumerate(src) if (len(s) > 1)]

        src = [src[i][:2000] for i in idxs]
        src = src[:1000]

        if (len(src) < 3):
            return None

        src_txt = [' '.join(sent) for sent in src]
        text = ' [SEP] [CLS] '.join(src_txt)
        src_subtokens = self.tokenizer.tokenize(text)
        #src_subtokens = src_subtokens[:4094]  ## 512가 최대인데 [SEP], [CLS] 2개 때문에 510
        src_subtokens = ['[CLS]'] + src_subtokens + ['[SEP]']

        src_subtoken_idxs = self.tokenizer.convert_tokens_to_ids(src_subtokens)
        _segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == self.sep_vid]
        segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
        segments_ids = []
        for i, s in enumerate(segs):
            if (i % 2 == 0):
                segments_ids += s * [0]
            else:
                segments_ids += s * [1]
        cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == self.cls_vid]
        labels = None
        src_txt = [original_src_txt[i] for i in idxs]
        tgt_txt = None
        
        return src_subtoken_idxs, labels, segments_ids, cls_ids, src_txt, tgt_txt

##### Network

In [6]:
class PositionalEncoding(nn.Module):

    def __init__(self, dropout, dim, max_len=5000):
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp((torch.arange(0, dim, 2, dtype=torch.float) *
                              -(math.log(10000.0) / dim)))
        pe[:, 0::2] = torch.sin(position.float() * div_term)
        pe[:, 1::2] = torch.cos(position.float() * div_term)
        pe = pe.unsqueeze(0)
        super(PositionalEncoding, self).__init__()
        self.register_buffer('pe', pe)
        self.dropout = nn.Dropout(p=dropout)
        self.dim = dim

    def forward(self, emb, step=None):
        emb = emb * math.sqrt(self.dim)
        if (step):
            emb = emb + self.pe[:, step][:, None, :]

        else:
            emb = emb + self.pe[:, :emb.size(1)]
        emb = self.dropout(emb)
        return emb

    def get_emb(self, emb):
        return self.pe[:, :emb.size(1)]

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, heads, d_ff, dropout):
        super(TransformerEncoderLayer, self).__init__()

        self.self_attn = MultiHeadedAttention(
            heads, d_model, dropout=dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, iter, query, inputs, mask):
        if (iter != 0):
            input_norm = self.layer_norm(inputs)
        else:
            input_norm = inputs

        mask = mask.unsqueeze(1)
        context = self.self_attn(input_norm, input_norm, input_norm,
                                 mask=mask)
        out = self.dropout(context) + inputs
        return self.feed_forward(out)

class ExtTransformerEncoder(nn.Module):
    def __init__(self, d_model, d_ff, heads, dropout, num_inter_layers=0):
        super(ExtTransformerEncoder, self).__init__()
        self.d_model = d_model
        self.num_inter_layers = num_inter_layers
        self.pos_emb = PositionalEncoding(dropout, d_model)
        self.transformer_inter = nn.ModuleList(
            [TransformerEncoderLayer(d_model, heads, d_ff, dropout)
             for _ in range(num_inter_layers)])
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.wo = nn.Linear(d_model, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, top_vecs, mask):
        """ See :obj:`EncoderBase.forward()`"""

        batch_size, n_sents = top_vecs.size(0), top_vecs.size(1)
        pos_emb = self.pos_emb.pe[:, :n_sents]
        x = top_vecs * mask[:, :, None].float()
        x = x + pos_emb

        for i in range(self.num_inter_layers):
            x = self.transformer_inter[i](i, x, x, ~ mask)  # all_sents * max_tokens * dim

        x = self.layer_norm(x)
        sent_scores = self.sigmoid(self.wo(x))
        sent_scores = sent_scores.squeeze(-1) * mask.float()

        return sent_scores
    
# class Bert(nn.Module):
#     temp_dir = 'D:/KoBertSum/temp'
#     def __init__(self, large, temp_dir, finetune=False):
#         super(Bert, self).__init__()
#         self.model = BertModel.from_pretrained("monologg/kobert", cache_dir=temp_dir)

#         self.finetune = finetune

#     def forward(self, x, segs, mask):
#         top_vec = self.model(input_ids=x, token_type_ids=segs, attention_mask=mask)[0]

#         return top_vec
    
class BigBird(nn.Module):
    def __init__(self, temp_dir, finetune=False):
        super(BigBird, self).__init__()
        self.model = AutoModel.from_pretrained("monologg/kobigbird-bert-base") # 모델 변경
        self.finetune = finetune

    def forward(self, x, mask):
        if self.finetune:
            outputs = self.model(x, attention_mask=mask)
            top_vec = outputs.last_hidden_state
        else:
            self.eval()
            with torch.no_grad():
                outputs = self.model(x, attention_mask=mask)
                top_vec = outputs.last_hidden_state
        return top_vec 
    
class ExtSummarizer(nn.Module):
    def __init__(self, args, device, checkpoint):
        super(ExtSummarizer, self).__init__()
        self.args = args
        self.device = device
        #self.bert = Bert(args.large, args.temp_dir)
        self.roberta = AutoModel.from_pretrained("monologg/kobigbird-bert-base")
        #self.bert = BigBird(args.large, args.temp_dir)

        # self.ext_layer = ExtTransformerEncoder(self.bert.model.config.hidden_size, args.ext_ff_size, args.ext_heads,
        #                                        args.ext_dropout, args.ext_layers)
        self.ext_layer = ExtTransformerEncoder(self.roberta.config.hidden_size, args.ext_ff_size, args.ext_heads,
                                               args.ext_dropout, args.ext_layers)
        # if (args.encoder == 'baseline'):
        #     bert_config = BertConfig(self.bert.model.config.vocab_size, hidden_size=args.ext_hidden_size,
        #                              num_hidden_layers=args.ext_layers, num_attention_heads=args.ext_heads, intermediate_size=args.ext_ff_size)
        #     self.bert.model = BertModel(bert_config)
        #     self.ext_layer = Classifier(self.bert.model.config.hidden_size)

        if(args.max_pos>512):
            my_pos_embeddings = nn.Embedding(args.max_pos, self.bert.model.config.hidden_size)
            my_pos_embeddings.weight.data[:512] = self.bert.model.embeddings.position_embeddings.weight.data
            my_pos_embeddings.weight.data[512:] = self.bert.model.embeddings.position_embeddings.weight.data[-1][None,:].repeat(args.max_pos-512,1)
            self.bert.model.embeddings.position_embeddings = my_pos_embeddings


        if checkpoint is not None:
            self.load_state_dict(checkpoint['model'], strict=True)
        else:
            if args.param_init != 0.0:
                for p in self.ext_layer.parameters():
                    p.data.uniform_(-args.param_init, args.param_init)
            if args.param_init_glorot:
                for p in self.ext_layer.parameters():
                    if p.dim() > 1:
                        xavier_uniform_(p)

        self.to(device)

    def forward(self, src, segs, clss, mask_src, mask_cls):
        top_vec = self.roberta(src)[0]
        #print(top_vec)
        #top_vec = top_vec.last_hidden_state
        sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
        sents_vec = sents_vec * mask_cls[:, :, None].float()
        sent_scores = self.ext_layer(sents_vec, mask_cls).squeeze(-1)
        return sent_scores, mask_cls

In [7]:
def summarize(text):
    
    def txt2input(text):
        #data = list(filter(None, text.split('\n')))
        #data = split_sentences(text)
        bertdata = BertData()
        txt_data = bertdata.preprocess(text)
        data_dict = {"src":txt_data[0],
                    "labels":[0,1,2],
                    "segs":txt_data[2],
                    "clss":txt_data[3],
                    "src_txt":txt_data[4],
                    "tgt_txt":None}
        input_data = []
        input_data.append(data_dict)
        return input_data
    
    input_data = txt2input(text)
    device = torch.device("cuda")
    
    def _pad(data, pad_id, width=-1):
        if (width == -1):
            width = max(len(d) for d in data)
        rtn_data = [d + [pad_id] * (width - len(d)) for d in data]
        return rtn_data
    
    pre_src = [x['src'] for x in input_data]
    pre_segs = [x['segs'] for x in input_data]
    pre_clss = [x['clss'] for x in input_data]

    src = torch.tensor(_pad(pre_src, 0)).cuda()
    segs = torch.tensor(_pad(pre_segs, 0)).cuda()
    mask_src = ~(src == 0)

    clss = torch.tensor(_pad(pre_clss, -1)).cuda()
    mask_cls = ~(clss == -1)
    clss[clss == -1] = 0

    clss.to(device).long()
    mask_cls.to(device).long()
    segs.to(device).long()
    mask_src.to(device).long()
    
    # clss.to(device)
    # mask_cls.to(device)
    # segs.to(device)
    # mask_src.to(device)
    
    #checkpoint = torch.load("D:/KoBertSum/ext/models/model_step_26000.pt")  # V2
    #checkpoint = torch.load("D:/KoBertSum/ext/models/model_step_5000.pt")  # V1
    #checkpoint = torch.load("D:/KoBertSum/ext/models/model_step_5000_2.pt")  # V3 - max_pos 1024
    checkpoint = torch.load("C:/Users/ADMIN/Desktop/dl업로드용/Kobertsum_0.3v/ext/models/1016_0350/model_step_27000.pt")  # BigBird
    model = ExtSummarizer(args, device, checkpoint)
    model.eval()

    with torch.no_grad():
        sent_scores, mask = model(src, segs, clss, mask_src, mask_cls)
        sent_scores = sent_scores + mask.float()
        sent_scores = sent_scores.cpu().data.numpy()
        print(sent_scores)
        selected_ids = np.argsort(-sent_scores, 1)
        print(selected_ids)
    
    return selected_ids

##### 결과 확인

In [64]:
text = '''
서울 서초구 초등학교 교사 사망에 대한 교육당국 합동조사 결과가 지난 4일 나왔지만 새롭게 규명된 것이 없다는 교사들의 반발이 거세다. 교육부와 서울시교육청이 사건 직후 사실관계 확인에 나서겠다며 합동조사에 착수했지만, 대부분 의혹을 경찰 수사로 미뤄두면서 ‘용두사미 조사’가 되고 말았다는 것이다. 전국초등교사노조는 “‘새내기 교사의 죽음에 무거운 책임감을 느낀다’며 내놓은 결과라고 납득할 수 없을 정도로 허술하다. 합동조사단이 해야 할 일은 해당 학교가 낸 가정통신문 내용의 사실 확인이 아닌 고인의 업무상 고충을 면면히 공개하는 것이어야 했다”며 재조사를 촉구했다. 전국 교사 4만명은 지난 주말에도 철저한 진상규명을 촉구하는 집회를 이어갔다.
실제로 합동조사 결과의 대부분은 학교 쪽이 고인의 사망 직후 냈던 입장문과 언론보도에서 제기된 내용이 맞는지 확인하는 수준에 그쳤다. 올해 3월 이후 고인의 학급 담임 교체 사실이 없었다거나, 해당 학급에서 올해 학교폭력 신고가 없었으며, 학급 내 이른바 ‘연필사건’ 이후 고인이 학부모로부터 전화를 받았다는 등의 단순 사실 확인에 불과하다. 연필사건 학생의 학부모가 고인의 휴대전화 번호를 알게 된 경위나 담임 자격 시비와 같은 폭언이 있었는지 여부, 학교 쪽이 연필사건을 원만히 중재했다고 한 7월13일 이후에도 추가적인 학부모 민원이 있었는지 등 정작 규명이 필요한 의혹에 대해서는 새롭게 밝혀낸 것이 하나도 없다.
무엇보다 합동조사 결과에는 고인이 사망에 이르기까지 학교나 학교장의 책임은 없었는지에 대한 내용이 쏙 빠져 있다. 고인은 연필사건 학생뿐 아니라 다른 2명의 부적응 학생으로 인한 고충이 적지 않았고, 모두 10차례나 학교 쪽에 학생 지도의 어려움을 이유로 상담 요청을 한 바 있다. 그러나 학교 쪽은 고인에게 얼른 전화번호를 바꾸라거나 학부모에게 심리검사 또는 상담을 받을 것을 권유하라고 조언하는 정도에 그쳤다. 학교가 함께 문제를 해결하려는 의지를 보이는 대신 개별 교사에게 책임을 지운 정황이 아닐 수 없다. 심지어 학교 쪽은 지난달 최초 작성한 입장문에서 연필사건이 원만히 해결된 것처럼 언급했다가 해당 내용을 삭제했는데, 애초 어떤 의도로 작성한 것인지 규명될 필요가 있다. 교육당국은 재발 방지 대책을 촘촘하게 마련하는 일도 중요하지만 제대로 된 진상규명이 우선이라는 점을 명심해야 한다.
서울 서초구 초등학교 교사 사망에 대한 교육당국 합동조사 결과가 지난 4일 나왔지만 새롭게 규명된 것이 없다는 교사들의 반발이 거세다. 교육부와 서울시교육청이 사건 직후 사실관계 확인에 나서겠다며 합동조사에 착수했지만, 대부분 의혹을 경찰 수사로 미뤄두면서 ‘용두사미 조사’가 되고 말았다는 것이다. 전국초등교사노조는 “‘새내기 교사의 죽음에 무거운 책임감을 느낀다’며 내놓은 결과라고 납득할 수 없을 정도로 허술하다. 합동조사단이 해야 할 일은 해당 학교가 낸 가정통신문 내용의 사실 확인이 아닌 고인의 업무상 고충을 면면히 공개하는 것이어야 했다”며 재조사를 촉구했다. 전국 교사 4만명은 지난 주말에도 철저한 진상규명을 촉구하는 집회를 이어갔다.
실제로 합동조사 결과의 대부분은 학교 쪽이 고인의 사망 직후 냈던 입장문과 언론보도에서 제기된 내용이 맞는지 확인하는 수준에 그쳤다. 올해 3월 이후 고인의 학급 담임 교체 사실이 없었다거나, 해당 학급에서 올해 학교폭력 신고가 없었으며, 학급 내 이른바 ‘연필사건’ 이후 고인이 학부모로부터 전화를 받았다는 등의 단순 사실 확인에 불과하다. 연필사건 학생의 학부모가 고인의 휴대전화 번호를 알게 된 경위나 담임 자격 시비와 같은 폭언이 있었는지 여부, 학교 쪽이 연필사건을 원만히 중재했다고 한 7월13일 이후에도 추가적인 학부모 민원이 있었는지 등 정작 규명이 필요한 의혹에 대해서는 새롭게 밝혀낸 것이 하나도 없다.
무엇보다 합동조사 결과에는 고인이 사망에 이르기까지 학교나 학교장의 책임은 없었는지에 대한 내용이 쏙 빠져 있다. 고인은 연필사건 학생뿐 아니라 다른 2명의 부적응 학생으로 인한 고충이 적지 않았고, 모두 10차례나 학교 쪽에 학생 지도의 어려움을 이유로 상담 요청을 한 바 있다. 그러나 학교 쪽은 고인에게 얼른 전화번호를 바꾸라거나 학부모에게 심리검사 또는 상담을 받을 것을 권유하라고 조언하는 정도에 그쳤다. 학교가 함께 문제를 해결하려는 의지를 보이는 대신 개별 교사에게 책임을 지운 정황이 아닐 수 없다. 심지어 학교 쪽은 지난달 최초 작성한 입장문에서 연필사건이 원만히 해결된 것처럼 언급했다가 해당 내용을 삭제했는데, 애초 어떤 의도로 작성한 것인지 규명될 필요가 있다. 교육당국은 재발 방지 대책을 촘촘하게 마련하는 일도 중요하지만 제대로 된 진상규명이 우선이라는 점을 명심해야 한다.
서울 서초구 초등학교 교사 사망에 대한 교육당국 합동조사 결과가 지난 4일 나왔지만 새롭게 규명된 것이 없다는 교사들의 반발이 거세다. 교육부와 서울시교육청이 사건 직후 사실관계 확인에 나서겠다며 합동조사에 착수했지만, 대부분 의혹을 경찰 수사로 미뤄두면서 ‘용두사미 조사’가 되고 말았다는 것이다. 전국초등교사노조는 “‘새내기 교사의 죽음에 무거운 책임감을 느낀다’며 내놓은 결과라고 납득할 수 없을 정도로 허술하다. 합동조사단이 해야 할 일은 해당 학교가 낸 가정통신문 내용의 사실 확인이 아닌 고인의 업무상 고충을 면면히 공개하는 것이어야 했다”며 재조사를 촉구했다. 전국 교사 4만명은 지난 주말에도 철저한 진상규명을 촉구하는 집회를 이어갔다.
실제로 합동조사 결과의 대부분은 학교 쪽이 고인의 사망 직후 냈던 입장문과 언론보도에서 제기된 내용이 맞는지 확인하는 수준에 그쳤다. 올해 3월 이후 고인의 학급 담임 교체 사실이 없었다거나, 해당 학급에서 올해 학교폭력 신고가 없었으며, 학급 내 이른바 ‘연필사건’ 이후 고인이 학부모로부터 전화를 받았다는 등의 단순 사실 확인에 불과하다. 연필사건 학생의 학부모가 고인의 휴대전화 번호를 알게 된 경위나 담임 자격 시비와 같은 폭언이 있었는지 여부, 학교 쪽이 연필사건을 원만히 중재했다고 한 7월13일 이후에도 추가적인 학부모 민원이 있었는지 등 정작 규명이 필요한 의혹에 대해서는 새롭게 밝혀낸 것이 하나도 없다.
무엇보다 합동조사 결과에는 고인이 사망에 이르기까지 학교나 학교장의 책임은 없었는지에 대한 내용이 쏙 빠져 있다. 고인은 연필사건 학생뿐 아니라 다른 2명의 부적응 학생으로 인한 고충이 적지 않았고, 모두 10차례나 학교 쪽에 학생 지도의 어려움을 이유로 상담 요청을 한 바 있다. 그러나 학교 쪽은 고인에게 얼른 전화번호를 바꾸라거나 학부모에게 심리검사 또는 상담을 받을 것을 권유하라고 조언하는 정도에 그쳤다. 학교가 함께 문제를 해결하려는 의지를 보이는 대신 개별 교사에게 책임을 지운 정황이 아닐 수 없다. 심지어 학교 쪽은 지난달 최초 작성한 입장문에서 연필사건이 원만히 해결된 것처럼 언급했다가 해당 내용을 삭제했는데, 애초 어떤 의도로 작성한 것인지 규명될 필요가 있다. 교육당국은 재발 방지 대책을 촘촘하게 마련하는 일도 중요하지만 제대로 된 진상규명이 우선이라는 점을 명심해야 한다.

'''
print('입력 길이',len(text))

입력 길이 3485


In [65]:
import kss
text = kss.split_sentences(text)
text

['서울 서초구 초등학교 교사 사망에 대한 교육당국 합동조사 결과가 지난 4일 나왔지만 새롭게 규명된 것이 없다는 교사들의 반발이 거세다.',
 '교육부와 서울시교육청이 사건 직후 사실관계 확인에 나서겠다며 합동조사에 착수했지만, 대부분 의혹을 경찰 수사로 미뤄두면서 ‘용두사미 조사’가 되고 말았다는 것이다.',
 '전국초등교사노조는 “‘새내기 교사의 죽음에 무거운 책임감을 느낀다’며 내놓은 결과라고 납득할 수 없을 정도로 허술하다. 합동조사단이 해야 할 일은 해당 학교가 낸 가정통신문 내용의 사실 확인이 아닌 고인의 업무상 고충을 면면히 공개하는 것이어야 했다”며 재조사를 촉구했다.',
 '전국 교사 4만명은 지난 주말에도 철저한 진상규명을 촉구하는 집회를 이어갔다.',
 '실제로 합동조사 결과의 대부분은 학교 쪽이 고인의 사망 직후 냈던 입장문과 언론보도에서 제기된 내용이 맞는지 확인하는 수준에 그쳤다.',
 '올해 3월 이후 고인의 학급 담임 교체 사실이 없었다거나, 해당 학급에서 올해 학교폭력 신고가 없었으며, 학급 내 이른바 ‘연필사건’ 이후 고인이 학부모로부터 전화를 받았다는 등의 단순 사실 확인에 불과하다.',
 '연필사건 학생의 학부모가 고인의 휴대전화 번호를 알게 된 경위나 담임 자격 시비와 같은 폭언이 있었는지 여부, 학교 쪽이 연필사건을 원만히 중재했다고 한 7월13일 이후에도 추가적인 학부모 민원이 있었는지 등 정작 규명이 필요한 의혹에 대해서는 새롭게 밝혀낸 것이 하나도 없다.',
 '무엇보다 합동조사 결과에는 고인이 사망에 이르기까지 학교나 학교장의 책임은 없었는지에 대한 내용이 쏙 빠져 있다.',
 '고인은 연필사건 학생뿐 아니라 다른 2명의 부적응 학생으로 인한 고충이 적지 않았고, 모두 10차례나 학교 쪽에 학생 지도의 어려움을 이유로 상담 요청을 한 바 있다.',
 '그러나 학교 쪽은 고인에게 얼른 전화번호를 바꾸라거나 학부모에게 심리검사 또는 상담을 받을 것을 권유하라고 조언하는 정도에 그쳤다.',
 '학교가 함께 문제를 해결

In [66]:
  def check_token(text):
        #data = list(filter(None, text.split('\n')))
        #data = split_sentences(text)
        bertdata = BertData()
        txt_data = bertdata.preprocess(text)
        Embedding_scr = txt_data[0]
        print('입력 받은 문장 길이:',len(text))
        print(Embedding_scr)
        print('총 임베딩 길이:',len(Embedding_scr))
        return Embedding_scr

In [67]:
check_token_size = check_token(text)

입력 받은 문장 길이: 39
[2, 3417, 3686, 3417, 3941, 2631, 3941, 2968, 4223, 2629, 2629, 3388, 3388, 3153, 3621, 2889, 4224, 2629, 3714, 2885, 2632, 4228, 2930, 3789, 3388, 2596, 2618, 2542, 3833, 2763, 524, 3732, 2760, 3667, 3833, 3145, 3400, 3099, 2585, 2653, 3190, 2936, 2579, 3729, 3614, 2873, 2856, 2629, 3388, 2963, 3728, 3245, 3247, 3729, 2572, 3429, 2873, 518, 3, 2, 2629, 3714, 3308, 3661, 3417, 3686, 3506, 2629, 3714, 3931, 3729, 3388, 2574, 3834, 4293, 3388, 3510, 2620, 2603, 4280, 3731, 3621, 2760, 3417, 2590, 2873, 3185, 4228, 2930, 3789, 3388, 3621, 3903, 3468, 4239, 3833, 3145, 516, 2889, 3308, 3310, 3728, 4271, 3723, 2601, 3906, 3468, 3388, 3094, 3230, 3115, 2942, 3187, 3417, 700, 3681, 2942, 3388, 3230, 3789, 3388, 701, 2542, 2935, 2605, 3148, 3580, 2873, 2856, 2579, 3729, 2873, 518, 3, 2, 3768, 2632, 3941, 2968, 2629, 3388, 2821, 3789, 2856, 702, 700, 3400, 2776, 2666, 2629, 3388, 3728, 3809, 3725, 3621, 3203, 2572, 3685, 3914, 3735, 2548, 3723, 2854, 2755, 2873, 701, 3185, 2776,

In [16]:
# data_dict = {"src":txt_data[0],  'src': [2,3417,..., 3, 2,...., 3]
#             "labels":[0,1,2],     labels': [0, 1, 2],
#             "segs":txt_data[2],  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ....]
#             "clss":txt_data[3],     'clss': [0, 59, 134, 254, 290, 349, 439, 558, 609, 680, 739, 782, 856],      
#             "src_txt":txt_data[4],        '서 울   서 초 구   초 등 학 교   교 사   사 망 에 
#             "tgt_txt":None}

None


In [42]:
input_data = txt2input(text)

In [43]:
input_data[0]

{'src': [2,
  3417,
  3686,
  3417,
  3941,
  2631,
  3941,
  2968,
  4223,
  2629,
  2629,
  3388,
  3388,
  3153,
  3621,
  2889,
  4224,
  2629,
  3714,
  2885,
  2632,
  4228,
  2930,
  3789,
  3388,
  2596,
  2618,
  2542,
  3833,
  2763,
  524,
  3732,
  2760,
  3667,
  3833,
  3145,
  3400,
  3099,
  2585,
  2653,
  3190,
  2936,
  2579,
  3729,
  3614,
  2873,
  2856,
  2629,
  3388,
  2963,
  3728,
  3245,
  3247,
  3729,
  2572,
  3429,
  2873,
  518,
  3,
  2,
  2629,
  3714,
  3308,
  3661,
  3417,
  3686,
  3506,
  2629,
  3714,
  3931,
  3729,
  3388,
  2574,
  3834,
  4293,
  3388,
  3510,
  2620,
  2603,
  4280,
  3731,
  3621,
  2760,
  3417,
  2590,
  2873,
  3185,
  4228,
  2930,
  3789,
  3388,
  3621,
  3903,
  3468,
  4239,
  3833,
  3145,
  516,
  2889,
  3308,
  3310,
  3728,
  4271,
  3723,
  2601,
  3906,
  3468,
  3388,
  3094,
  3230,
  3115,
  2942,
  3187,
  3417,
  700,
  3681,
  2942,
  3388,
  3230,
  3789,
  3388,
  701,
  2542,
  2935,
  2605,
  3148,

In [28]:
result = summarize(text)

[[1.6601198 1.2734241 1.1563554 1.0941479 1.0885135 1.0911074 1.096595
  1.1044112 1.1063108 1.1072062 1.1194654 1.1539675 1.2159421]]
[[ 0  1 12  2 11 10  9  8  7  6  3  5  4]]


In [31]:
len(text)

13

In [30]:
result[0]

array([ 0,  1, 12,  2, 11, 10,  9,  8,  7,  6,  3,  5,  4], dtype=int64)

In [29]:
[text[i] for i in result[0][:len(text)//3]]
#[text[i] for i in result[0][:3]]

['서울 서초구 초등학교 교사 사망에 대한 교육당국 합동조사 결과가 지난 4일 나왔지만 새롭게 규명된 것이 없다는 교사들의 반발이 거세다.',
 '교육부와 서울시교육청이 사건 직후 사실관계 확인에 나서겠다며 합동조사에 착수했지만, 대부분 의혹을 경찰 수사로 미뤄두면서 ‘용두사미 조사’가 되고 말았다는 것이다.',
 '교육당국은 재발 방지 대책을 촘촘하게 마련하는 일도 중요하지만 제대로 된 진상규명이 우선이라는 점을 명심해야 한다.',
 '전국초등교사노조는 “‘새내기 교사의 죽음에 무거운 책임감을 느낀다’며 내놓은 결과라고 납득할 수 없을 정도로 허술하다. 합동조사단이 해야 할 일은 해당 학교가 낸 가정통신문 내용의 사실 확인이 아닌 고인의 업무상 고충을 면면히 공개하는 것이어야 했다”며 재조사를 촉구했다.']

##### 탐구

In [ ]:
text = ['BNK경남은행 간부의 562억원 횡령 사건이 채 잊히기도 전인데 또 다른 금융사고가 연발하고 있다.',
'수법도 거래 기업의 미공개 정보로 주식을 거래해 부당이득을 취하고, 고객 몰래 계좌를 개설하는 등 눈앞의 이익을 위해 온갖 불법·편법 행위가 동원되고 있다.',
'KB국민은행의 직원들이 고객사의 미공개정보를 이용해 주식을 사고팔아 127억원의 부당이득을 챙긴 사실이 최근 금융당국에 적발됐다.',
'이들은 2021년 1월부터 올해 4월까지 61개 상장사의 증권 업무를 대행하며 알게 된 무상증자 규모와 일정을 주식 매수에 이용했다.',
'정보 공개 전 미리 주식을 사뒀다가 공시 뒤 주가가 오르면 팔았다.',
'무상증자를 하게 되면 기업재무구조가 건전한 것으로 풀이돼 주가에는 호재로 작용한다.',
'이런 방식으로 66억원 정도를 챙겼고, 일부는 다른 부서의 동료나 가족 등에게도 정보를 전달했다.',
'이 과정에서도 61억원 상당의 부당 이득이 발생했다.',
'대구은행 일부 지점 직원 수십명은 평가 실적을 올리기 위해 지난해 1000여건이 넘는 고객 문서를 위조해 증권 계좌를 개설한 것으로 파악됐다.',
'이 직원들은 내점한 고객을 상대로 증권사 연계 계좌를 만들어 달라고 요청한 뒤 해당 계좌 신청서를 복사해 고객의 동의 없이 같은 증권사의 계좌를 하나 더 만들었다.',
'A증권사 위탁 계좌 개설 신청서를 받고, 같은 신청서를 복사해 A증권사 해외선물계좌까지 개설하는 방식이다.',
'최근 한 고객이 동의하지 않은 계좌가 개설됐다는 사실을 알게 돼 대구은행에 민원을 제기하면서 직원들의 비리가 드러났다.',
'대구은행은 문제를 인지하고도 금감원에 이 사실을 보고하지 않았고, 영업점들에 공문을 보내 불건전 영업행위를 예방하라고 안내하는 데 그쳤다.',
'금융실명제법 위반, 사문서 위조 등에 해당할 수 있는 범죄행위를 대수롭지 않게 넘기는 안일함이 혀를 차게 한다.',
'국내 은행은 땅 짚고 헤엄치기식 이자장사로 평균 1억원대 고연봉을 누리는 직종이다.',
'시중은행은 미국발 고금리에 편승해 거둬들인 막대한 예대마진으로 최근 수년간 성과급 잔치를 벌여 국민의 눈총을 받았다.',
'국민의 재산으로 손쉽게 수익을 올리는 직종이라면 누구보다 엄격한 도덕적 기준을 세워도 모자랄 판에 ‘내 몫을 더 챙기겠다’며 이기적 탐욕을 부리고 있으니 말문이 막힌다.',
'자체 내부통제가 안 된다면 현행 솜방망이 처벌 수위를 높이는 수밖에 없다.',
'주요 동기가 경제적 이익인 만큼 벌금이나 과징금, 양형 부과 수준을 크게 높여 법 무서운 줄 알도록 해야 한다.',
'주요 선진국에서 이미 도입한 불공정거래 범죄자에 대한 자본시장 거래제한제도도 적극 검토할 필요가 있다.']

In [ ]:
text = ' '.join(text)
text

In [ ]:
import kss

array = []
text = kss.split_sentences(text)
text

In [ ]:
src = []
src.append(text)
src

In [ ]:
len(src)

In [ ]:
original_src_txt = [''.join(s) for s in text]
original_src_txt

In [ ]:
len(original_src_txt)

In [ ]:
idxs = [i for i, s in enumerate(text) if (len(s) > 1)]
idxs

In [ ]:
text = [text[i][:2000] for i in idxs]
text

In [ ]:
text = text[:1000]
len(text)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("monologg/kobert", do_lower_case=True)

In [ ]:
src_txt = [''.join(sent) for sent in text]
src_txt

In [ ]:
text = ' [SEP] [CLS] '.join(src_txt)
text

In [ ]:
src_subtokens = tokenizer.tokenize(text)
src_subtokens

In [ ]:
vocab = tokenizer.get_vocab()
print(vocab.get('▁('))
print([tokenizer.encode(token) for token in src_subtokens])

In [ ]:
src_subtokens[0]

In [ ]:
src_subtokens = src_subtokens[:510]  ## 512가 최대인데 [SEP], [CLS] 2개 때문에 510
len(src_subtokens)

In [ ]:
src_subtokens = ['[CLS]'] + src_subtokens + ['[SEP]']
len(src_subtokens)

In [ ]:
src_subtoken_idxs = tokenizer.convert_tokens_to_ids(src_subtokens)
src_subtoken_idxs

In [ ]:
len(text) # 원문 문장 개수

In [ ]:
src_subtoken_idxs.count(2)  # 2는 [CLS] 토큰
# 즉 9개의 문장에 해당하는 [CLS] 토큰만 입력된다는 이야기.

In [ ]:
 self.sep_vid = self.tokenizer.convert_tokens_to_ids(self.sep_token)

_segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == self.sep_vid]
segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
segments_ids = []
for i, s in enumerate(segs):
    if (i % 2 == 0):
        segments_ids += s * [0]
    else:
        segments_ids += s * [1]
cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == self.cls_vid]

In [ ]:
text = '''
일본 정부가 문재인 대통령이 도쿄올림픽을 계기로 일본을 방문하는 방향으로 한일 양국이 조율하고 있다는 15일 요미우리신문의 보도를 부인했다.

정부 대변인인 가토 가쓰노부(加藤勝信) 관방장관은 이날 오전 정례 기자회견에서 관련 질문에 “말씀하신 보도와 같은 사실이 없는 것으로 안다”고 밝혔다.

앞서 요미우리는 한국 측이 도쿄올림픽을 계기로 한 문 대통령의 방일을 타진했고, 일본 측은 수용하는 방향이라고 이날 보도했다.

한국 측은 문 대통령의 방일 때 스가 요시히데(菅義偉) 총리와 처음으로 정상회담을 하겠다는 생각이라고 요미우리는 전했다.

가토 장관은 한일 정상회담에 대한 일본 정부의 자세에 대해 “그런 사실이 없기 때문에 가정의 질문에 대해 답하는 것을 삼가겠다”고 말했다.

그는 한국 측의 독도방어훈련에 ‘어떤 대항 조치를 생각하고 있느냐’는 질문에는 “한국 해군의 훈련에 대해 정부로서는 강한 관심을 가지고 주시하는 상황이어서 지금 시점에선 논평을 삼가겠다”고 말을 아꼈다.

가토 장관은 “다케시마(竹島·일본이 주장하는 독도의 명칭)는 역사적 사실에 비춰봐도, 국제법상으로도 명백한 일본 고유의 영토”라며 독도 영유권 주장을 되풀이했다.

그러면서 “다케시마 문제에 대해서는 계속 우리나라의 영토, 영해, 영공을 단호히 지키겠다는 결의로 냉정하고 의연하게 대응해갈 생각”이라고 밝혔다.
'''

In [ ]:
data = list(filter(None, text.split('\n')))
data

In [ ]:
pwd

In [ ]:
from prepro.tokenization_kobert import KoBertTokenizer

In [ ]:
class BertData():
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained("monologg/kobert", do_lower_case=True)

        self.sep_token = '[SEP]'
        self.cls_token = '[CLS]'
        self.pad_token = '[PAD]'
        # self.sep_vid = self.tokenizer.token2idx[self.sep_token]
        # self.cls_vid = self.tokenizer.token2idx[self.cls_token]
        # self.pad_vid = self.tokenizer.token2idx[self.pad_token]

        self.sep_vid = self.tokenizer.convert_tokens_to_ids(self.sep_token)
        self.cls_vid = self.tokenizer.convert_tokens_to_ids(self.cls_token)
        self.pad_vid = self.tokenizer.convert_tokens_to_ids(self.pad_token)

    def preprocess(self, src):

        if (len(src) == 0):
            return None

        original_src_txt = [' '.join(s) for s in src]

        labels = [0] * len(src)
        # for l in oracle_ids:
        #     labels[l] = 1

        idxs = [i for i, s in enumerate(src) if (len(s) > 1)]

        src = [src[i][:1000] for i in idxs]
        #labels = [labels[i] for i in idxs]
        src = src[:2000]
        #labels = labels[:self.args.max_nsents]

        # if (len(src) < 1):
        #     return None
        # if (len(labels) == 0):
        #     return None

        src_txt = [' '.join(sent) for sent in src]
        # text = [' '.join(ex['src_txt'][i].split()[:self.args.max_src_ntokens]) for i in idxs]
        # text = [_clean(t) for t in text]
        text = ' [SEP] [CLS] '.join(src_txt)
        src_subtokens = self.tokenizer.tokenize(text)
        src_subtokens = src_subtokens[:510]
        src_subtokens = ['[CLS]'] + src_subtokens + ['[SEP]']

        src_subtoken_idxs = self.tokenizer.convert_tokens_to_ids(src_subtokens)
        _segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == self.sep_vid]
        segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
        segments_ids = []
        for i, s in enumerate(segs):
            if (i % 2 == 0):
                segments_ids += s * [0]
            else:
                segments_ids += s * [1]
        cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == self.cls_vid]
        #labels = labels[:len(cls_ids)]

        #tgt_txt = '<q>'.join([' '.join(tt) for tt in tgt])
        src_txt = [original_src_txt[i] for i in idxs]
        return src_subtoken_idxs, segments_ids, cls_ids, src_txt

In [ ]:
def txt2input(text):
    data = list(filter(None, text.split('\n')))
    #data = split_sentences(text)
    bertdata = BertData()
    txt_data = bertdata.preprocess(data)
    data_dict = {"src":txt_data[0],
                "segs":txt_data[1],
                "clss":txt_data[2],
                "src_txt":txt_data[3]}
    input_data = []
    input_data.append(data_dict)
    return input_data

In [ ]:
input_data = txt2input(text)
input_data[0]['src'].count(2)

In [ ]:
inputs = txt2input(text)
len(inputs[0]['src']), len(inputs[0]['clss']), len(inputs[0]['segs'])

In [ ]:
device = torch.device("cuda")

def _pad(data, pad_id, width=-1):
    if (width == -1):
        width = max(len(d) for d in data)
    rtn_data = [d + [pad_id] * (width - len(d)) for d in data]
    return rtn_data

pre_src = [x['src'] for x in inputs]
pre_segs = [x['segs'] for x in inputs]
pre_clss = [x['clss'] for x in inputs]

src = torch.tensor(_pad(pre_src, 0))
segs = torch.tensor(_pad(pre_segs, 0))
mask_src = ~(src == 0)

clss = torch.tensor(_pad(pre_clss, -1))
mask_cls = ~(clss == -1)
clss[clss == -1] = 0

clss.to(device).long()
mask_cls.to(device).long()
segs.to(device).long()
mask_src.to(device).long()

# checkpoint = torch.load("D:/KoBertSum/ext/models/model_step_26000.pt")
# model = ExtSummarizer(args, device, checkpoint)
# model.eval()

# with torch.no_grad():
#     sent_scores, mask = model(src, segs, clss, mask_src, mask_cls)
#     print(sent_scores)
#     sent_scores = sent_scores + mask.float()
#     sent_scores = sent_scores.cpu().data.numpy()
#     print(sent_scores)
#     selected_ids = np.argsort(-sent_scores, 1)
#     print(selected_ids)

In [ ]:
model = BertModel.from_pretrained("monologg/kobert")
top_vec = model(input_ids=src, token_type_ids=segs, attention_mask=mask_src)[0]
top_vec

In [ ]:
src.size(), clss.size(), segs.size()

In [ ]:
clss

In [ ]:
top_vec[torch.arange(top_vec.size(0)).unsqueeze(1)]

In [ ]:
sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
sents_vec

In [ ]:
sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
sents_vec = sents_vec * mask_cls[:, :, None].float()
sent_scores = self.ext_layer(sents_vec, mask_cls).squeeze(-1)